In [1]:
import math
import scipy.stats
import numpy

In [2]:
def getData(direction):
    sample_distribution_file = open(direction, "r")
    data = [float(x.replace(",", ".")) for x in sample_distribution_file.read().splitlines()]
    return data

In [19]:
path = "../continuous/data/data_dagum_4P.txt"
data = getData(path)

In [4]:
def num_bins_doane(data):
    N = len(data)
    skewness = scipy.stats.skew(data)
    sigma_g1 = math.sqrt((6 * (N - 2)) / ((N + 1) * (N + 3)))
    num_bins = 1 + math.log(N, 2) + math.log(1 + abs(skewness) / sigma_g1, 2)
    return math.ceil(num_bins)

def num_bins_freedman_diaconis(data):
    iqr = numpy.percentile(data, 75) - numpy.percentile(data, 25)
    bin_width = 2 * iqr / (len(data) ** (1/3))
    num_bins = (max(data) - min(data)) / bin_width
    return math.ceil(num_bins)


def num_bins_scott(data):
    sigma = numpy.std(data)
    bin_width = 3.5 * sigma / (len(data) ** (1 / 3))
    num_bins = (max(data) - min(data)) / bin_width
    return math.ceil(num_bins)


def num_bins_stone(data):
    return math.ceil((2 * len(data)/3)  ** (1 / 3))


def num_bins_rice(data):
    return math.ceil(2 * (len(data) ** (1 / 3)))


def num_bins_sturges(data):
    return math.ceil(numpy.log2(len(data)) + 1)


def num_bins_sqrt(data):
    return math.ceil(numpy.sqrt(len(data)))


In [20]:
import numpy as np

def leave_one_out_error(data, num_bins, left_out_index):
    """Calcula el error de dejar un dato fuera para un número específico de bins."""
    # Excluye el dato en left_out_index
    data_without_left_out = np.delete(data, left_out_index)
    
    # Crea un histograma con el conjunto de datos excluyendo el dato
    hist, bin_edges = np.histogram(data_without_left_out, bins=num_bins)
    
    # Encuentra el bin correspondiente al dato excluido
    bin_index = np.digitize(data[left_out_index], bin_edges) - 1
    bin_index = min(bin_index, len(hist) - 1)  # Asegura que el índice esté dentro de los límites
    
    # Estima la densidad para el dato excluido dividiendo el conteo del bin por el ancho del bin
    estimated_density = hist[bin_index] / (bin_edges[bin_index+1] - bin_edges[bin_index])
    
    # Calcula el error cuadrático para el dato excluido
    actual_density = 1  # dado que es un solo dato
    squared_error = (estimated_density - actual_density) ** 2
    
    return squared_error

def optimal_bins_cross_validation(data, max_bins=50):
    """Encuentra el número óptimo de bins utilizando validación cruzada leave-one-out."""
    total_errors = []
    
    # Prueba con diferentes números de bins
    for num_bins in range(1, max_bins + 1):
        total_error_for_bins = sum([leave_one_out_error(data, num_bins, i) for i in range(len(data))])
        total_errors.append(total_error_for_bins)
    
    # Devuelve el número de bins que minimiza el error
    return np.argmin(total_errors) + 1


print(optimal_bins_cross_validation(data))  # Devuelve el número óptimo de bins para el conjunto de datos

1


In [5]:
## max("fd", "sturges")
len(numpy.histogram_bin_edges(data, bins="auto"))

67

##### **Freedman Diaconis Estimator**
$$
h = 2 \frac{IQR}{n^{1/3}}
$$

In [6]:
len(numpy.histogram_bin_edges(data, bins="fd")) - 1, num_bins_freedman_diaconis(data)

(66, 66)

##### **Doane**
$$
n_h = 1 + \log_{2}(n) +\log_{2}\left(1 + \frac{|g_1|}{\sigma_{g_1}}\right)\\
g_1 = E\left[\left(\frac{x - \mu}{\sigma}\right)^3\right]\\
\sigma_{g_1} = \sqrt{\frac{6(n - 2)}{(n + 1)(n + 3)}}
$$

In [7]:
len(numpy.histogram_bin_edges(data, bins="doane")) - 1, num_bins_doane(data)

(18, 18)

##### **Scott**
$$
h = \sigma \sqrt[3]{\frac{24 \sqrt{\pi}}{n}}
$$

In [8]:
len(numpy.histogram_bin_edges(data, bins="scott")) - 1, num_bins_scott(data)

(39, 38)

##### **Stone**
$$

$$

In [9]:
len(numpy.histogram_bin_edges(data, bins="stone")) - 1, num_bins_stone(data)

(40, 12)

##### **Rice**
$$
n_h = 2n^{1/3}
$$

In [10]:
len(numpy.histogram_bin_edges(data, bins="rice")) - 1, num_bins_rice(data)

(26, 26)

##### **Sturges**
$$
n_h = \log _{2}(n) + 1
$$

In [11]:
len(numpy.histogram_bin_edges(data, bins="sturges")) - 1, num_bins_sturges(data)

(13, 13)

##### **Sqrt**
$$
n_h = \sqrt n
$$

In [12]:
len(numpy.histogram_bin_edges(data, bins="sqrt")) - 1, num_bins_sqrt(data)

(47, 47)